## 수정사항
#### 2021.06.04
* Gripper 자유도 45도로 제한됨에 따라 파지점 방향 수정, 바닥 파지면은 삭제

## set running directory

In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [2]:
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir


DATA_PATH = os.path.join(os.environ['RNB_PLANNING_DIR'], "data")
try_mkdir(DATA_PATH)

TEST_DATA_PATH = os.path.join(DATA_PATH, "test-pairwise")
try_mkdir(TEST_DATA_PATH)

In [3]:
VISUALIZE = False
TIMEOUT_REACH = 30
TIMEOUT_RETRIEVE = 30
TIMEOUT_SELF = 3
TOOL_RPY = (-np.pi/2,0,0)
SHOW_PERIOD = 0.01

ROBOT_TYPE = RobotType.indy7

if ROBOT_TYPE == RobotType.indy7:
    ROBOT_NAME = "indy0"
    TOOL_LINK = "indy0_tcp"
    TOOL_XYZ = (0,0,0.14)
    GRIP_DEPTH = 0.05
    HOME_POSE = (0,0,0,0,0,0)
elif ROBOT_TYPE == RobotType.panda:
    ROBOT_NAME = "panda0"
    TOOL_LINK = "panda0_hand"
    TOOL_XYZ = (0,0,0.112)
    GRIP_DEPTH = 0.03
    HOME_POSE = (0,-0.3,0,-0.5,0,2.5,0)
else:
    raise(NotImplementedError("Unknown robot type"))
    
    
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, ROBOT_TYPE.name)
try_mkdir(ROBOT_DATA_ROOT)

## init combined robot config

In [4]:

from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, ROBOT_TYPE, None,
                INDY_IP)]
              , connection_list=[False])

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# # deprecated: s_builder.reset_reference_coord(ref_name="floor")

connection command:
indy0: False


## get ghnd with detected robot config

In [5]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {ROBOT_NAME: ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob, start_rviz=VISUALIZE)
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


In [6]:
from pkg.utils.joint_utils import get_tf
shoulder_link = gscene.urdf_content.joint_map[gscene.joint_names[1]].child
shoulder_height = get_tf(shoulder_link, HOME_DICT, gscene.urdf_content)[2,3]

## add environment

In [7]:
from pkg.geometry.geometry import *
gtems_robot = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

## init planning scene

In [8]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

## Register binders

In [9]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [10]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name=TOOL_LINK, 
                 dims=(0.01,)*3, center=TOOL_XYZ, rpy=TOOL_RPY, color=(1,0,0,1), display=True, collision=False, fixed=True)
gripper = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0))

## planner

In [11]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
from pkg.planning.filtering.pair_svm import PairSVM


gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
lcheck = LatticedChecker(pscene, gcheck)
pcheck = PairSVM(pscene)

g_checker = MoveitPlanner(pscene, motion_filters = [gcheck])
gr_checker = MoveitPlanner(pscene, motion_filters = [gcheck, rcheck])
grl_checker = MoveitPlanner(pscene, motion_filters = [gcheck, rcheck, lcheck])
p_checker =  MoveitPlanner(pscene, motion_filters = [pcheck])

## ui

In [12]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


# Object Classes

In [13]:
from pkg.planning.filtering.lattice_model import *
from pkg.planning.filtering.lattice_model.scene_building import *
        
PLANE_THICKNESS = 0.1
CLEARANCE = 1e-3

##
# @class WorkPlane
# @brief working plane. target and obstacle objects are generated on this plane
class WorkPlane(ObstacleBase):
    RTH_MIN = (0.3, -np.pi/2, -0.2)
    RTH_MAX = (0.3, +np.pi/2, 0.5)
    RPY_MIN = (0, 0, -np.pi/6)
    RPY_MAX = (0, 0, +np.pi/6)
    DIM_MIN = (0.6, 0.6, PLANE_THICKNESS)
    DIM_MAX = (0.6, 0.6, PLANE_THICKNESS)
    GTYPE = GEOTYPE.BOX
    COLOR=  (0.8,0.8,0.2,0.5)
    
    def __init__(self, gscene, name, *args, **kwargs):
        ObstacleBase.__init__(self, gscene, name, *args, **kwargs)
        
    def is_overlapped_with(self, gtem):
        verts, radii = gtem.get_vertice_radius()
        verts_global = np.add(np.matmul(verts, gtem.orientation_mat.transpose()), gtem.center)
        verts_wp = np.multiply(DEFAULT_VERT_DICT[self.GTYPE], tuple(self.DIM[:2])+(self.H,))
        verts_wp_global = np.add(np.matmul(verts_wp, self.geometry.orientation_mat.transpose()), 
                                 np.add(self.geometry.center, (0,0,self.H/2)))
        return get_gjk_distance(get_point_list(verts_global), get_point_list(verts_wp_global))-radii < 1e-4
    
##
# @class PlaneObstacle
# @brief Obstacles on the workplane
class PlaneObject(ObstacleBase):
    RTH_MIN = (0.2, -np.pi/2, -0.2)
    RTH_MAX = (0.8, +np.pi/2, +0.5)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.02, GRIP_DEPTH, GRIP_DEPTH)
    DIM_MAX = (0.055, 0.3, 0.3)
    GTYPE = GEOTYPE.BOX
    COLOR =  (0.2,0.2,0.8,0.5)
    def __init__(self, gscene, name, workplane, GRIP_DEPTH, CLEARANCE, XYZ_LOC=None, **kwargs):
        self.GRIP_DEPTH = GRIP_DEPTH
        self.DIM_MIN = (0.02, GRIP_DEPTH, GRIP_DEPTH)
        ObstacleBase.__init__(self, gscene=gscene, name=name, **kwargs)
        verts, radii = self.geometry.get_vertice_radius()
        verts_rot = np.matmul(self.geometry.orientation_mat, verts.transpose()) ## verices with global orientaion
        verts_rot_loc = np.matmul(workplane.geometry.Toff[:3,:3].transpose(), verts_rot) ## verices with local orientaion
        max_verts = np.max(verts_rot_loc, axis=-1)
        min_verts = np.min(verts_rot_loc, axis=-1)
        if XYZ_LOC is None:
            self.XYZ_LOC = np.random.uniform(np.negative(workplane.DIM)/2-min_verts+radii,np.array(workplane.DIM)/2-max_verts-radii)
            self.XYZ_LOC[2] = workplane.DIM[2]/2 + self.DIM[2]/2 + CLEARANCE
        else:
            self.XYZ_LOC = self.XYZ_LOC
        self.XYZ = np.matmul(workplane.geometry.Toff[:3,:3], self.XYZ_LOC) + workplane.geometry.Toff[:3,3]
        self.RPY = (0,0,0)
        self.geometry.set_offset_tf(center = self.XYZ, orientation_mat=Rot_rpy(self.RPY))
        self.RTH = cart2cyl(*self.XYZ)
        gscene.update_marker(self.geometry)
        
        


            
def disperse_objects(gscene, object_class, key, Nobj, workplane_on, GRIP_DEPTH, CLEARANCE=1e-3):
    clear_class(gscene, key, Nobj)
        
    obs_list = []
    for iw in range(Nobj):
        while len(obs_list) != iw+1:
            obs = object_class(gscene, "{}_{}".format(key, iw), workplane_on, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)
            remove_this = False
            for obs_pre in obs_list:
                if obs_pre.is_overlapped_with(obs.geometry):
                    remove_this = True
                    break
            if remove_this:
                gscene.remove(obs.geometry)
            else:
                obs_list.append(obs)
    return obs_list


def add_object(pscene, obj, HANDLE_THICKNESS=1e-6, HANDLE_COLOR = (1,0,0,0.3)):
    gscene = pscene.gscene
    handles = []
    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_tp_a", link_name="base_link", 
                       dims=(obj.DIM[1], GRIP_DEPTH, HANDLE_THICKNESS), center=(0,0,obj.DIM[2]/2-GRIP_DEPTH/2), rpy=(np.pi/2,0,np.pi/2), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_tp_b", link_name="base_link", 
                       dims=(obj.DIM[1], GRIP_DEPTH, HANDLE_THICKNESS), center=(0,0,obj.DIM[2]/2-GRIP_DEPTH/2), rpy=(np.pi/2,0,-np.pi/2), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_ft_a", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,obj.DIM[1]/2-GRIP_DEPTH/2,0), rpy=(0,np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_ft_b", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,obj.DIM[1]/2-GRIP_DEPTH/2,0), rpy=(0,-np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_bk_a", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,-obj.DIM[1]/2+GRIP_DEPTH/2,0), rpy=(-np.pi,-np.pi/2,0), 
                       color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                   parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_bk_b", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,-obj.DIM[1]/2+GRIP_DEPTH/2,0), rpy=(-np.pi,+np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    action_points_dict = {"placement": PlacePoint("placement", obj.geometry, [0,0,-obj.DIM[2]/2], [0,0,0])}
    grasp_points = {handle.name: Grasp2Point(handle.name, handle, None, (0,0,0)) for handle in handles}
    for k, ap in grasp_points.items():
        ap.get_redundancy = lambda : {'w': (0, 0), 'x': (0, 0), 'y': (0, 0), 'z': (0, 0)}
    action_points_dict.update(grasp_points)
    obj_pscene = pscene.create_subject(oname=obj.name, gname=obj.name, _type=CustomObject, 
                                 action_points_dict=action_points_dict)
    return obj_pscene, handles

In [14]:
N_max_sample = 100
N_print = 5
N_sample_single_obj = 1

   Use a production WSGI server instead.
 * Debug mode: off


# Load and verify

In [15]:
N_MAX = 1000


gtimer = GlobalTimer.instance()
gtimer.reset(stack=True)
DATASET_LIST = sorted(os.listdir(ROBOT_DATA_ROOT))
N_tot = 0
N_suc = 0
labels = []
g_results = []
gr_results = []
grl_results = []
p_results = []
print("DATASET_LIST: {}".format(DATASET_LIST))
gtimer.tic("full_loop")
for DATASET_DIR in DATASET_LIST:
    DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DATASET_DIR)
    print("DATASET_PATH: {}".format(DATASET_PATH))
    DATA_LIST = sorted(os.listdir(DATASET_PATH))
    print("DATA_LIST: {}".format(DATA_LIST))
    for DATA_FILE in DATA_LIST:
        DATA_PATH = os.path.join(DATASET_PATH, DATA_FILE)
        print("DATA_PATH: {}".format(DATA_PATH))
        data = load_pickle(DATA_PATH)
        gtem_args = data["gtem_args"]
        obj_args = data["obj_args"]
        samples = data["samples"]
        gtem_remove = []
        for gtem in gscene:
            if gtem.link_name == "base_link" and gtem.parent is None:
                gtem_remove.append(gtem)
        for gtem in gtem_remove:
            gscene.remove(gtem)
                
        gid_list = np.arange(len(gtem_args)).tolist()
        for gidx in gid_list:
            args = gtem_args[gidx]
            if args['parent'] is not None:
                if args['parent'] not in gscene.NAME_DICT:
                    gid_list.append(gidx)
                    continue
            gscene.create_safe(**args)
            
        pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)
            
        for i_s, sample in enumerate(samples):
            obj_name, from_state, to_state, redundancy_dict, \
                success_kin_, success_reach_, success_retrieve_, \
                Traj_kin_, Traj_reach_, Traj_retrieve_, \
                time_kin_, time_reach_, time_retrieve_ = sample
            
            obj, obj_arg = DummyObject(), obj_args[obj_name]
            obj.name = obj_name
            obj.GRIP_DEPTH = obj_arg["GRIP_DEPTH"]
            obj.DIM = obj_arg["DIM"]
            obj.geometry = gscene.NAME_DICT[obj_arg["gname"]]
            obj_pscene, handles = add_object(pscene, obj)
            
            gscene.update()
#             g_checker.update_gscene(only_self_collision=True)
#             gr_checker.update_gscene(only_self_collision=True)
#             grl_checker.update_gscene(only_self_collision=True)
                            
            pscene.set_object_state(from_state)

            with gtimer.block("Grasp"):
                success_g = \
                    g_checker.plan_transition(from_state=from_state, to_state=to_state, redundancy_dict=redundancy_dict, 
                                               test_filters_only=True, timeout=TIMEOUT_SELF)

            with gtimer.block("GraspReach"):
                success_gr = \
                    gr_checker.plan_transition(from_state=from_state, to_state=to_state, redundancy_dict=redundancy_dict, 
                                               test_filters_only=True, timeout=TIMEOUT_SELF)

            with gtimer.block("GRL"):
                success_grl = \
                    grl_checker.plan_transition(from_state=from_state, to_state=to_state, redundancy_dict=redundancy_dict, 
                                               test_filters_only=True, timeout=TIMEOUT_SELF)
            with gtimer.block("GRL"):
                success_p = \
                    p_checker.plan_transition(from_state=from_state, to_state=to_state, redundancy_dict=redundancy_dict, 
                                               test_filters_only=True, timeout=TIMEOUT_SELF)

            labels.append(success_retrieve_)
            g_results.append(success_g)
            gr_results.append(success_gr)
            grl_results.append(success_grl)
            p_results.append(success_p)
                
            pscene.set_object_state(from_state)
            pscene.remove_subject(obj_pscene.oname)
            for handle in handles:
                gscene.remove(handle)  
            gscene.update_markers_all()  
            N_tot += 1
            N_suc += success_retrieve_
            if N_tot>=N_MAX:
                break
        print("GRL: {} / SUCCESS: {} / TOTAL: {} / ELAPSED TIME: {} s / ETA: {} s".format(
            np.sum(grl_results), N_suc, N_tot, *np.round(np.array(gtimer.eta("full_loop", N_tot, N_MAX))/1000, 1)))
        if N_tot>=N_MAX:
            break
    if N_tot>=N_MAX:
        break

DATASET_LIST: ['20210611-093216', '20210611-093559', '20210611-093945', '20210611-094206', '20210611-094418', '20210611-094654', '20210611-094947', '20210611-095224', '20210611-095458', '20210611-095737']
DATASET_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-093216
DATA_LIST: ['00001.pkl', '00002.pkl', '00003.pkl', '00004.pkl', '00005.pkl', '00006.pkl', '00007.pkl', '00008.pkl', '00009.pkl', '00010.pkl', '00011.pkl', '00012.pkl', '00013.pkl', '00014.pkl', '00015.pkl', '00016.pkl', '00017.pkl', '00018.pkl', '00019.pkl', '00020.pkl', '00021.pkl', '00022.pkl', '00023.pkl', '00024.pkl', '00025.pkl', '00026.pkl', '00027.pkl', '00028.pkl', '00029.pkl', '00030.pkl', '00031.pkl', '00032.pkl', '00033.pkl', '00034.pkl', '00035.pkl', '00036.pkl', '00037.pkl', '00038.pkl', '00039.pkl', '00040.pkl', '00041.pkl', '00042.pkl', '00043.pkl', '00044.pkl', '00045.pkl', '00046.pkl', '00047.pkl', '00048.pkl', '00049.pkl', '00050.pkl']
DATA_PATH: /home/rnb/Projects/rnb-planning/dat

GRL: 29 / SUCCESS: 28 / TOTAL: 96 / ELAPSED TIME: 2.1 s / ETA: 21.5 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-093216/00049.pkl
GRL: 29 / SUCCESS: 28 / TOTAL: 98 / ELAPSED TIME: 2.1 s / ETA: 21.3 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-093216/00050.pkl
GRL: 30 / SUCCESS: 29 / TOTAL: 100 / ELAPSED TIME: 2.2 s / ETA: 21.6 s
DATASET_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-093559
DATA_LIST: ['00001.pkl', '00002.pkl', '00003.pkl', '00004.pkl', '00005.pkl', '00006.pkl', '00007.pkl', '00008.pkl', '00009.pkl', '00010.pkl', '00011.pkl', '00012.pkl', '00013.pkl', '00014.pkl', '00015.pkl', '00016.pkl', '00017.pkl', '00018.pkl', '00019.pkl', '00020.pkl', '00021.pkl', '00022.pkl', '00023.pkl', '00024.pkl', '00025.pkl', '00026.pkl', '00027.pkl', '00028.pkl', '00029.pkl', '00030.pkl', '00031.pkl', '00032.pkl', '00033.pkl', '00034.pkl', '00035.pkl', '00036.pkl', '00037.pkl', '00038.pkl', '00039.p

GRL: 68 / SUCCESS: 60 / TOTAL: 194 / ELAPSED TIME: 4.4 s / ETA: 22.8 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-093559/00048.pkl
GRL: 69 / SUCCESS: 61 / TOTAL: 196 / ELAPSED TIME: 4.5 s / ETA: 22.9 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-093559/00049.pkl
GRL: 70 / SUCCESS: 62 / TOTAL: 198 / ELAPSED TIME: 4.5 s / ETA: 22.9 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-093559/00050.pkl
GRL: 71 / SUCCESS: 63 / TOTAL: 200 / ELAPSED TIME: 4.6 s / ETA: 22.9 s
DATASET_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-093945
DATA_LIST: ['00001.pkl', '00002.pkl', '00003.pkl', '00004.pkl', '00005.pkl', '00006.pkl', '00007.pkl', '00008.pkl', '00009.pkl', '00010.pkl', '00011.pkl', '00012.pkl', '00013.pkl', '00014.pkl', '00015.pkl', '00016.pkl', '00017.pkl', '00018.pkl', '00019.pkl', '00020.pkl', '00021.pkl', '00022.pkl', '00023.pkl', '00024.pkl', '00025.pkl', '00026.pkl

GRL: 106 / SUCCESS: 98 / TOTAL: 288 / ELAPSED TIME: 7.1 s / ETA: 24.5 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-093945/00045.pkl
GRL: 106 / SUCCESS: 98 / TOTAL: 290 / ELAPSED TIME: 7.1 s / ETA: 24.5 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-093945/00046.pkl
GRL: 106 / SUCCESS: 98 / TOTAL: 292 / ELAPSED TIME: 7.1 s / ETA: 24.3 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-093945/00047.pkl
GRL: 107 / SUCCESS: 99 / TOTAL: 294 / ELAPSED TIME: 7.2 s / ETA: 24.3 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-093945/00048.pkl
GRL: 108 / SUCCESS: 100 / TOTAL: 296 / ELAPSED TIME: 7.2 s / ETA: 24.3 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-093945/00049.pkl
GRL: 108 / SUCCESS: 100 / TOTAL: 298 / ELAPSED TIME: 7.2 s / ETA: 24.2 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-093945/00050.pkl
GR

GRL: 137 / SUCCESS: 129 / TOTAL: 390 / ELAPSED TIME: 9.5 s / ETA: 24.3 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-094206/00046.pkl
GRL: 138 / SUCCESS: 129 / TOTAL: 392 / ELAPSED TIME: 9.5 s / ETA: 24.3 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-094206/00047.pkl
GRL: 140 / SUCCESS: 131 / TOTAL: 394 / ELAPSED TIME: 9.6 s / ETA: 24.4 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-094206/00048.pkl
GRL: 141 / SUCCESS: 132 / TOTAL: 396 / ELAPSED TIME: 9.7 s / ETA: 24.4 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-094206/00049.pkl
GRL: 141 / SUCCESS: 132 / TOTAL: 398 / ELAPSED TIME: 9.7 s / ETA: 24.4 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-094206/00050.pkl
GRL: 141 / SUCCESS: 132 / TOTAL: 400 / ELAPSED TIME: 9.7 s / ETA: 24.3 s
DATASET_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-094418
DATA_

...

GRL: 200 / SUCCESS: 187 / TOTAL: 574 / ELAPSED TIME: 13.5 s / ETA: 23.5 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-094654/00038.pkl
GRL: 201 / SUCCESS: 188 / TOTAL: 576 / ELAPSED TIME: 13.5 s / ETA: 23.5 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-094654/00039.pkl
GRL: 201 / SUCCESS: 188 / TOTAL: 578 / ELAPSED TIME: 13.6 s / ETA: 23.5 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-094654/00040.pkl
GRL: 201 / SUCCESS: 188 / TOTAL: 580 / ELAPSED TIME: 13.6 s / ETA: 23.4 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-094654/00041.pkl
GRL: 202 / SUCCESS: 189 / TOTAL: 582 / ELAPSED TIME: 13.6 s / ETA: 23.4 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-094654/00042.pkl
GRL: 203 / SUCCESS: 190 / TOTAL: 584 / ELAPSED TIME: 13.7 s / ETA: 23.4 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-094654/00

GRL: 244 / SUCCESS: 227 / TOTAL: 668 / ELAPSED TIME: 16.3 s / ETA: 24.3 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-094947/00035.pkl
GRL: 245 / SUCCESS: 228 / TOTAL: 670 / ELAPSED TIME: 16.3 s / ETA: 24.3 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-094947/00036.pkl
GRL: 246 / SUCCESS: 229 / TOTAL: 672 / ELAPSED TIME: 16.3 s / ETA: 24.3 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-094947/00037.pkl
GRL: 247 / SUCCESS: 230 / TOTAL: 674 / ELAPSED TIME: 16.4 s / ETA: 24.3 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-094947/00038.pkl
GRL: 248 / SUCCESS: 231 / TOTAL: 676 / ELAPSED TIME: 16.5 s / ETA: 24.4 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-094947/00039.pkl
GRL: 248 / SUCCESS: 231 / TOTAL: 678 / ELAPSED TIME: 16.5 s / ETA: 24.3 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-094947/00

GRL: 275 / SUCCESS: 259 / TOTAL: 760 / ELAPSED TIME: 18.6 s / ETA: 24.5 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-095224/00031.pkl
GRL: 277 / SUCCESS: 261 / TOTAL: 762 / ELAPSED TIME: 18.7 s / ETA: 24.6 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-095224/00032.pkl
GRL: 278 / SUCCESS: 262 / TOTAL: 764 / ELAPSED TIME: 18.8 s / ETA: 24.6 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-095224/00033.pkl
GRL: 279 / SUCCESS: 263 / TOTAL: 766 / ELAPSED TIME: 18.9 s / ETA: 24.7 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-095224/00034.pkl
GRL: 280 / SUCCESS: 264 / TOTAL: 768 / ELAPSED TIME: 19.0 s / ETA: 24.7 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-095224/00035.pkl
GRL: 282 / SUCCESS: 266 / TOTAL: 770 / ELAPSED TIME: 19.1 s / ETA: 24.8 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-095224/00

GRL: 310 / SUCCESS: 293 / TOTAL: 850 / ELAPSED TIME: 21.1 s / ETA: 24.8 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-095458/00026.pkl
GRL: 310 / SUCCESS: 293 / TOTAL: 852 / ELAPSED TIME: 21.1 s / ETA: 24.8 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-095458/00027.pkl
GRL: 312 / SUCCESS: 295 / TOTAL: 854 / ELAPSED TIME: 21.2 s / ETA: 24.8 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-095458/00028.pkl
GRL: 313 / SUCCESS: 296 / TOTAL: 856 / ELAPSED TIME: 21.3 s / ETA: 24.9 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-095458/00029.pkl
GRL: 313 / SUCCESS: 296 / TOTAL: 858 / ELAPSED TIME: 21.3 s / ETA: 24.8 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-095458/00030.pkl
GRL: 313 / SUCCESS: 296 / TOTAL: 860 / ELAPSED TIME: 21.3 s / ETA: 24.8 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-095458/00

GRL: 336 / SUCCESS: 318 / TOTAL: 942 / ELAPSED TIME: 23.1 s / ETA: 24.6 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-095737/00022.pkl
GRL: 337 / SUCCESS: 319 / TOTAL: 944 / ELAPSED TIME: 23.2 s / ETA: 24.6 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-095737/00023.pkl
GRL: 337 / SUCCESS: 319 / TOTAL: 946 / ELAPSED TIME: 23.2 s / ETA: 24.6 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-095737/00024.pkl
GRL: 338 / SUCCESS: 320 / TOTAL: 948 / ELAPSED TIME: 23.3 s / ETA: 24.5 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-095737/00025.pkl
GRL: 338 / SUCCESS: 320 / TOTAL: 950 / ELAPSED TIME: 23.3 s / ETA: 24.5 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-095737/00026.pkl
GRL: 339 / SUCCESS: 320 / TOTAL: 952 / ELAPSED TIME: 23.3 s / ETA: 24.5 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/indy7/20210611-095737/00

In [16]:
print(gtimer)
print("")
print("=========== GraspChecker ==============")
print_confusion_mat(GT=labels, Res=g_results)
print("")
print("=========== GraspReacher ==============")
print_confusion_mat(GT=labels, Res=gr_results)
print("")
print("=========== TotalChecker ==============")
print_confusion_mat(GT=labels, Res=grl_results)
print("")
print("=========== Pair Checker ==============")
print_confusion_mat(GT=labels, Res=p_results)

full_loop: 	6065408.1 ms/500 = 12130.8 ms (57.734/24263.697)
Grasp: 	1647.4 ms/1000 = 1.6 ms (0.669/4.15)
GraspReach: 	2020.6 ms/1000 = 2.0 ms (0.686/5.239)
GRL: 	17265.5 ms/2000 = 8.6 ms (0.601/65.022)


=========== GraspChecker ==============
	 PP 	 	 PN 	 	 1000
GP 	 332 	 	 0 	 	 100.00%
GN 	 424 	 	 244 	 36.53%
AL 	 43.92% 	 100.00% 	 57.60%

=========== GraspReacher ==============
	 PP 	 	 PN 	 	 1000
GP 	 330 	 	 2 	 	 99.40%
GN 	 37 	 	 631 	 94.46%
AL 	 89.92% 	 99.68% 	 96.10%

=========== TotalChecker ==============
	 PP 	 	 PN 	 	 1000
GP 	 320 	 	 12 	 	 96.39%
GN 	 31 	 	 637 	 95.36%
AL 	 91.17% 	 98.15% 	 95.70%

=========== Pair Checker ==============
	 PP 	 	 PN 	 	 1000
GP 	 286 	 	 46 	 	 86.14%
GN 	 59 	 	 609 	 91.17%
AL 	 82.90% 	 92.98% 	 89.50%


# Indy7
```
=========== GraspChecker ==============
	 PP 	 	 PN 	 	 1000
GP 	 332 	 	 0 	 	 100.00%
GN 	 424 	 	 244 	 36.53%
AL 	 43.92% 	 100.00% 	 57.60%

=========== GraspReacher ==============
	 PP 	 	 PN 	 	 1000
GP 	 330 	 	 2 	 	 99.40%
GN 	 37 	 	 631 	 94.46%
AL 	 89.92% 	 99.68% 	 96.10%

=========== TotalChecker ==============
	 PP 	 	 PN 	 	 1000
GP 	 320 	 	 12 	 	 96.39%
GN 	 31 	 	 637 	 95.36%
AL 	 91.17% 	 98.15% 	 95.70%

=========== Pair Checker ==============
	 PP 	 	 PN 	 	 1000
GP 	 286 	 	 46 	 	 86.14%
GN 	 59 	 	 609 	 91.17%
AL 	 82.90% 	 92.98% 	 89.50%

```

# Panda
```
=========== GraspChecker ==============
	 PP 	 	 PN 	 	 1000
GP 	 350 	 	 0 	 	 100.00%
GN 	 391 	 	 259 	 39.85%
AL 	 47.23% 	 100.00% 	 60.90%

=========== GraspReacher ==============
	 PP 	 	 PN 	 	 1000
GP 	 340 	 	 10 	 	 97.14%
GN 	 76 	 	 574 	 88.31%
AL 	 81.73% 	 98.29% 	 91.40%

=========== TotalChecker ==============
	 PP 	 	 PN 	 	 1000
GP 	 338 	 	 12 	 	 96.57%
GN 	 71 	 	 579 	 89.08%
AL 	 82.64% 	 97.97% 	 91.70%

=========== Pair Checker ==============
	 PP 	 	 PN 	 	 1000
GP 	 234 	 	 116 	 	 66.86%
GN 	 48 	 	 602 	 92.62%
AL 	 82.98% 	 83.84% 	 83.60%
```